In [21]:
import cupy as cp
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from io import StringIO

neon_palette = ['#00FFFF', '#00FF00', '#FF00FF', '#FFFF00', '#008080', '#008000', '#800080', '#808000']

binance_csv = """Altcoin,Preço (USD),Volume (24h),Market Cap (USD),Variação (24h)
Aergo (AERGO),0.12,5000000,60000000,2.5
Aion (AION),0.08,3000000,40000000,-1.2
Bluzelle (BLZ),0.05,2000000,25000000,0.8
Celer Network (CELR),0.03,1500000,18000000,1.5
COTI (COTI),0.04,1000000,20000000,-0.5
Elrond (EGLD),50,8000000,2500000000,3.2
Fetch.ai (FET),0.25,6000000,125000000,1.8
Harmony (ONE),0.02,4000000,10000000,-0.9
Hedera Hashgraph (HBAR),0.06,2500000,30000000,0.7
ICON (ICX),0.30,7000000,150000000,2.1
IOST (IOST),0.01,1800000,50000000,-1.5
Kava (KAVA),0.80,3500000,400000000,0.5
Komodo (KMD),0.55,1500000,275000000,-0.2
Kyber Network (KNC),0.70,1000000,350000000,3.0
Nervos Network (CKB),0.008,8000000,40000000,-0.7
Ocean Protocol (OCEAN),0.45,1200000,225000000,1.8
Ontology (ONT),0.50,2000000,250000000,1.2
Quant (QNT),100,4000000,5000000000,-1.5
Ravencoin (RVN),0.035,3000000,175000000,2.1
Reserve (RSV),0.02,1500000,10000000,0.5"""

df_altcoins_binance = pd.read_csv(StringIO(binance_csv))

github_csv = """Altcoin,Número de Commits,Número de Issues Abertas,Número de Pull Requests,Atividade da Comunidade (escala 1-10)
Aergo (AERGO),180,30,60,7.5
Aion (AION),150,25,50,6.8
Bluzelle (BLZ),120,20,40,6.2
Celer Network (CELR),190,32,65,7.8
COTI (COTI),160,28,55,7.2
Elrond (EGLD),250,40,80,8.5
Fetch.ai (FET),220,35,70,8.2
Harmony (ONE),170,30,58,7.5
Hedera Hashgraph (HBAR),200,38,75,8.0
ICON (ICX),230,35,75,8.3
IOST (IOST),140,22,45,6.5
Kava (KAVA),210,35,70,8.0
Komodo (KMD),165,28,55,7.3
Kyber Network (KNC),240,40,80,8.7
Nervos Network (CKB),185,32,62,7.8
Ocean Protocol (OCEAN),205,35,70,8.2
Ontology (ONT),195,33,68,7.9
Quant (QNT),260,45,90,9.0
Ravencoin (RVN),175,30,60,7.6
Reserve (RSV),155,25,50,7.0"""

df_altcoins_github = pd.read_csv(StringIO(github_csv))

def plot_box_violin(df, x_col, y_col, title):
    fig = go.Figure()
    for i, val in enumerate(df[x_col].unique()):
        fig.add_trace(go.Violin(x=df[x_col][df[x_col] == val],
                                 y=df[y_col][df[x_col] == val],
                                 name=val,
                                 box_visible=True,
                                 meanline_visible=True,
                                 line_color=neon_palette[i % len(neon_palette)]))
    fig.update_layout(title=title, xaxis_title=x_col, yaxis_title=y_col,
                      plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

def plot_sunburst_chart(df, values, names, title):
    fig = px.sunburst(df, values=values, path=[names], title=title, color_discrete_sequence=neon_palette)
    fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

def plot_boxplot(df, x_col, y_col, title):
    fig = px.box(df, x=x_col, y=y_col, title=title, color_discrete_sequence=neon_palette)
    fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

def plot_correlation_matrix(df, title):
    numeric_df = df.select_dtypes(include=[np.number])
    corr_matrix = numeric_df.corr()
    fig = go.Figure(data=go.Heatmap(z=corr_matrix.values, x=corr_matrix.index,
                                   y=corr_matrix.columns, colorscale='Viridis'))
    fig.update_layout(title=title, xaxis_nticks=36,
                      plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

def plot_pca(df, title):
    x = df.select_dtypes(include=[np.number])
    x = StandardScaler().fit_transform(x)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data=principalComponents, columns=['Componente Principal 1', 'Componente Principal 2'])
    finalDf = pd.concat([principalDf, df[['Altcoin']]], axis=1)
    fig = px.scatter(finalDf, x='Componente Principal 1', y='Componente Principal 2', color='Altcoin',
                     title=title, color_discrete_sequence=neon_palette)
    fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

plot_sunburst_chart(df_altcoins_binance, 'Market Cap (USD)', 'Altcoin', 'Market Cap das Altcoins')
plot_boxplot(df_altcoins_binance, 'Altcoin', 'Volume (24h)', 'Distribuição do Volume (24h)')
plot_box_violin(df_altcoins_binance, 'Altcoin', 'Preço (USD)', 'Distribuição do Preço das Altcoins (USD)')
plot_boxplot(df_altcoins_binance, 'Altcoin', 'Variação (24h)', 'Distribuição da Variação (24h)')

fig = px.scatter(df_altcoins_binance, x="Preço (USD)", y="Volume (24h)", color="Altcoin",
                 size='Market Cap (USD)', hover_data=['Variação (24h)'], color_discrete_sequence=neon_palette)
fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white', title='Relação Preço-Volume')
fig.show()

fig = px.parallel_coordinates(df_altcoins_binance, color="Variação (24h)",
                             dimensions=["Preço (USD)", "Volume (24h)", "Market Cap (USD)", "Variação (24h)"],
                             color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=0)
fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white', title='Coordenadas Paralelas')
fig.show()

fig = go.Figure(go.Heatmap(z=df_altcoins_binance["Market Cap (USD)"], x=df_altcoins_binance["Preço (USD)"],
                           y=df_altcoins_binance["Volume (24h)"], colorscale="Viridis"))
fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white', title='Mapa de Calor de Densidade')
fig.show()

plot_box_violin(df_altcoins_github, 'Altcoin', 'Número de Commits', 'Número de Commits por Altcoin')
plot_boxplot(df_altcoins_github, 'Altcoin', 'Atividade da Comunidade (escala 1-10)', 'Atividade da Comunidade')
plot_correlation_matrix(df_altcoins_github, 'Matriz de Correlação - Dados GitHub')

fig = px.parallel_categories(df_altcoins_github, dimensions=['Altcoin', 'Número de Commits',
                                                            'Número de Issues Abertas', 'Número de Pull Requests'],
                             color="Atividade da Comunidade (escala 1-10)", color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white', title='Categorias Paralelas - GitHub')
fig.show()

plot_pca(df_altcoins_binance, 'Análise de Componentes Principais (PCA) - Binance')
plot_pca(df_altcoins_github, 'Análise de Componentes Principais (PCA) - GitHub')



In [36]:
!pip install together
!pip install openai
import re
import openai
import json

from together import Together

# Insira suas chaves API aqui
openai.api_key = "KEY"
together_api_key = "KEY"

client = Together(api_key=together_api_key)

def generate_candidate_prompts(task, prompt_example, response_example):
    messages = [
        {"role": "system", "content": """Você é um assistente de IA útil e inofensivo. Sua tarefa é gerar sete pares de prompt-resposta diversos e de alta qualidade para uma determinada tarefa, com base em um exemplo.

        **Regras:**
        1.  Garanta que os novos exemplos sejam exclusivos e cubram diferentes aspectos da tarefa.
        2.  Eles devem ser bem escritos e precisos.
        3.  Concentre-se em criar exemplos realistas e úteis.

        Responda neste formato:
        ```json
        [
          {
            "prompt": "COLOQUE_O_PROMPT_AQUI",
            "response": "COLOQUE_A_RESPOSTA_AQUI"
          },
          {
            "prompt": "COLOQUE_O_PROMPT_AQUI",
            "response": "COLOQUE_A_RESPOSTA_AQUI"
          },
          ...
        ]
        ```"""},
        {"role": "user", "content": f"""
        **Tarefa:** {task}

        **Exemplo:**

        **Prompt:**
        {prompt_example}

        **Resposta:**
        {response_example}
        """},
    ]

    completion = openai.ChatCompletion.create(  # Usando OpenAI para gerar exemplos
        model="gpt-4",
        messages=messages,
        max_tokens=4000,
        temperature=0.05
    )

    response_text = completion.choices[0].message.content

    try:
        prompts_and_responses = json.loads(response_text)
    except json.JSONDecodeError:
        print("Erro ao decodificar JSON. Texto da resposta:")
        print(response_text)
        prompts_and_responses = [] # Retorna lista vazia em caso de erro

    return prompts_and_responses

def generate_system_prompt(task, prompt_examples):
    messages = [
        {"role": "system", "content": """Você é um assistente de IA útil e inofensivo. Sua tarefa é criar um prompt de sistema conciso e eficaz para um modelo de linguagem grande, com base em uma determinada tarefa e um conjunto de exemplos de prompt-resposta.

        **Regras:**
        1. O prompt do sistema deve descrever com precisão a tarefa e o formato de saída desejado.
        2. Deve ser claro, conciso e fácil para o modelo de linguagem entender.
        3. Evite complexidade ou instruções desnecessárias.

        Responda neste formato:

        ```
        ## Análise de Altcoins para Investidor Moderado com Horizonte de Longo Prazo (5 anos)


**1. ETH (Ethereum)**

**Plataforma:** Ethereum

**Utilidade:** DeFi, NFTs, Web3

**Análise:**

A ETH é a segunda maior criptomoeda em market cap e possui um ecossistema vasto e em constante desenvolvimento. É considerada uma altcoin com grande potencial de crescimento no longo prazo, impulsionada pela adoção crescente de DeFi, NFTs e Web3.

**Recomendação:**

Comprar

**Justificativa:**

A ETH é uma escolha sólida para investidores de longo prazo que buscam crescimento de capital. O ecossistema Ethereum é robusto e possui uma grande comunidade de desenvolvedores e usuários.

**Sugestão de estratégia:**

Para a ETH, recomenda-se utilizar a estratégia DCA para reduzir o risco de entrada e aproveitar as possíveis flutuações de preço.


**2. SOL (Solana)**

**Plataforma:** Solana

**Utilidade:** DeFi, NFTs, Web3

**Análise:**

A SOL é uma altcoin que ganhou popularidade por sua alta velocidade de transação e baixas taxas. É considerada uma concorrente da Ethereum e possui um ecossistema em crescimento.

**Recomendação:**

Comprar (com alocação moderada)

**Justificativa:**

A SOL possui um grande potencial de crescimento, mas ainda é uma altcoin relativamente nova e com maior risco em comparação com a ETH.

**Sugestão de estratégia:**

Para a SOL, recomenda-se utilizar a estratégia DCA e monitorar o desenvolvimento do ecossistema Solana.


**3. ADA (Cardano)**

**Plataforma:** Cardano

**Utilidade:** DeFi, NFTs, Contratos inteligentes

**Análise:**

A ADA é uma altcoin que se destaca por sua abordagem científica e foco em segurança e escalabilidade. É considerada uma concorrente da Ethereum e possui um ecossistema em desenvolvimento.

**Recomendação:**

Comprar (com alocação moderada)

**Justificativa:**

A ADA possui um grande potencial de crescimento, mas ainda está em fase de desenvolvimento e sua adoção em massa ainda é incerta.

**Sugestão de estratégia:**

Para a ADA, recomenda-se utilizar a estratégia DCA e acompanhar o desenvolvimento do ecossistema Cardano.

        ```"""},
        {"role": "user", "content": f"""
        **Tarefa:** {task}

        **Exemplos de Prompt-Resposta:**
        ```json
        {json.dumps(prompt_examples, indent=2)}
        ```
        """},
    ]

    completion = openai.ChatCompletion.create(  # Usando OpenAI para gerar o prompt do sistema
        model="gpt-4",
        messages=messages,
        max_tokens=1000,
        temperature=0.05
    )

    response_text = completion.choices[0].message.content
    system_prompt = response_text.strip()

    return system_prompt

def test_llama(generated_examples, prompt_example, system_prompt):
    messages = [{"role": "system", "content": system_prompt}]

    for example in generated_examples:
        messages.append({"role": "user", "content": example['prompt']})
        messages.append({"role": "assistant", "content": example['response']})

    messages.append({"role": "user", "content": prompt_example.strip()})

    completion = client.chat.completions.create(
        model="meta-llama/Llama-2-70b-chat-hf",
        messages=messages,
        max_tokens=2000,
        temperature=0.05
    )

    response_text = completion.choices[0].message.content

    return response_text

# ... (Restante do código - task, prompt_example, response_example, debate, etc. - permanece o mesmo) ...